# SVM Spam Filter
#### Miloslav Homer, Marek Zpěváček

### Matematika

#### Objective function

Naším cieľom bude minimalizovať objective function:
$$
    J(\alpha) = \frac{1}{m}\sum_{i=1}^m\left[1-y^{(i)}K^{(i)\top}\alpha\right]_+ + \frac{\lambda}{2}\alpha^\top K\alpha,
$$
kde hľadáme $\alpha$, $m$ je počet správ, $\lambda$ je parameter, ktorý volíme na začiatku a $K$ je Gaussovský kernel, tj:
$$
    K(x,z)=\operatorname{exp}\left(-\frac{1}{2\tau^2}\|x-z\|_2^2\right).
$$
Značením $[t]_+$ rozumieme $\max{(t,0)}$.

### Parsing dát

#### Formát vstupu

Na vstupe dostaneme súbor obsahujúci (v tomto poradí):
počet emailov, dĺžku slovníka (tj počet rôznych slov vyskytujúcich sa v týchto emailoch), slovník (oddelené medzerou), zoznam emailov. Prvé číslo je vždy buď 0 alebo 1, indikuje či je daný email spam. Ďalej zoznam čísel ukončených -1, na $i$-tej pozícii sa nachádza číslo $j$, tj $i$-te slovo emailu je $j$-te slovo slovníka.

In [14]:
import numpy as np
#np.set_printoptions(threshold=np.nan)
import random
import math

In [15]:
def readData(path):
    reader=open(path)
    
    # ignore first line
    reader.readline()

    # second line contains number of emails and dictionary size
    array = reader.readline().split(' ')
    num_of_emails = int(array[0])
    dict_size = int(array[1])
    
    # ignore third line
    reader.readline()
    
    x= np.zeros((num_of_emails,dict_size), dtype=np.int)
    y= np.zeros(num_of_emails, dtype=np.int)
    
    # x[i,j] number of occurences of j-th word in i-th email
    # y[i] i-th email is spam?
    for i in range(num_of_emails):
        array=reader.readline().split(' ')
        int_array=[int(e) for e in array]
        y[i]=int_array[0]
        
        #indexing mind*uck - check encoding.txt file 
        index=0
        for j in range(1,int(len(array)/2)):
            index=index+int_array[2*j-1]
            x[i,index]=int_array[2*j]
    reader.close()
    return (x,y)

### Learning fáza

Potrebujeme zvoliť dve konštanty: počet krokov zostupu (`num_outer_loops`) a regularizačný parameter (`lam`). 
Regularizačný parameter označuje ako veľmi nám vadí keď daný email nespĺňa klasifikáciu a vôbec nie je jasné ako ho vybrať.
V zadaní písali $\lambda = \frac{1}{64m}$, tak im budeme veriť.

In [16]:
num_outer_loops = 40
lam = 64 #lambda is a reserved word in python

Najprv musíme vedieť spočítať Gaussian Kernel podľa vzorca:
$$
K(x,z)=\operatorname{exp}\left(-\frac{1}{2\tau^2}\|x-z\|_2^2\right).
$$

In [17]:
def gaussKernel(a,b):
    Ker=np.zeros((len(a),len(b)))
    for i in range(len(a)):
        for j in range(len(b)): 
            Ker[i,j]=np.exp(-(np.linalg.norm(a[i]-b[j],2))**2/(2*tau*tau))
    return Ker

Výuková fáza prebieha pomocou stochastického gradientového zostupu (stochastic gradient descent).
V každom kroku zvolíme náhodne smer zostupu (dá sa na to pozrieť aj tak, že si náhodne zvolíme dáta ktorými sa budeme zaoberať). 
Dôležitou súčasťou je tkz. learning rate (v tomto prípade $\frac{1}{\sqrt{i+1}}$ kde $i$ označuje číslo kroku (+1 preto, lebo počítame od nuly aby sme nedelili nulou)).

In [18]:
def learnSVM(x,y):
    #init
    m=len(y)
    x=1*(x>0)
    y=2*y-1
    alpha=np.zeros(m)
    avg_alpha = np.zeros(m)
    #compute kernel
    Ker = gaussKernel(x,x)
    #stochastic gradient descent
    for i in range(num_outer_loops * m):
        #choose a direction
        index = random.randint(0,m-1)
        #compute gradient g
        margin=y[index]*np.dot(Ker[index],alpha)
        g=np.dot(Ker[index],alpha[index])/(m*lam)-(margin<1)*y[index]*Ker[index]    
        #apply gradient
        alpha=alpha-g/math.sqrt(i+1)
        avg_alpha+=alpha
        
    avg_alpha=avg_alpha/(num_outer_loops*m)
    
    return avg_alpha

### Testovacia fáza

In [19]:
def testSVM(x_test,y_test,avg_alpha,x_train):
    #shorten vectors
    x_test = 1*(x_test>0)
    x_train = 1*(x_train>0)
    y_test=2*y_test-1
    #compute kernel
    Ker = gaussKernel(x_test,x_train)
    #decide
    preds = np.dot(Ker,avg_alpha)
    test_err=np.sum((np.multiply(preds,y_test))<=0)/len(y_test)
    return test_err

### Testy a výsledky

In [20]:
tau = 8
num_of_tests = 1
testSizes = ['50', '100', '200', '400', '800', '1400']
#testSizes = ['50','1400']

In [21]:
(m_test, category_test) = readData('spam_data/MATRIX.TEST')
for size in testSizes:
    err = 0
    for i in range(num_of_tests):
        (m_train, y_train) = readData('spam_data/MATRIX.TRAIN.' + size)
        avg_alpha = learnSVM(m_train, y_train)
        #print(avg_alpha)
        err += testSVM(m_test, category_test, avg_alpha, m_train)
        #print(err)
    err = err / num_of_tests
    print('Train size:', size, 'Error:', err)

Train size: 50 Error: 0.01125
Train size: 100 Error: 0.015
Train size: 200 Error: 0.0025
Train size: 400 Error: 0.00375
Train size: 800 Error: 0.0
Train size: 1400 Error: 0.0
